In [ ]:

# myslq 패키지 install
!pip install mysql-connector-python -q

In [ ]:
from mysql import connector
print(connector.__version__)

9.2.0


In [ ]:
args = {
      'host' : 'localhost',
      'user' : 'root',
      'password' : 'root1234',
      'port' : 3306
}

try:
    conn = connector.connect(**args)   # localhost에 접속 해서 접속 객체를 생성
    cursor = conn.cursor() # 실제 db에 명령을 전달(sql)
    print('DB success connection')
    cursor.execute('select * from world.city limit 10')
    # print(cursor.fetchall())   # sql 명령어을 통해 조회한 데이터를 객체로 가져옴
    result = []
    for id,name,countycode,district,population in cursor.fetchall():
        res = {'id':id, 'name':name, 'ct':countycode, 'dt':district, 'pp':population}
        result.append(res)
except Exception as e:
    print(e)
finally:
    if conn:
        conn.close()

DB success connection


In [ ]:
# table 생성
args = {
    'host' : 'localhost',
    'user' : 'root',
    'password' : 'root1234',
    'port' : 3306,
    # 'database' : 'student_db'   # 접속할 때 활성화할 db(스키마)명
}

try:
    conn = connector.connect(**args)   # localhost에 접속 해서 접속 객체를 생성
    cursor = conn.cursor() # 실제 db에 명령을 전달(sql)
    print('DB success connection')
    # db 생성
    cursor.execute('create database if not exists student_db')   # 미리 생성해야 파이썬에서 스키마 생성됨
    cursor.execute('use student_db')
    cursor.execute(
        '''
        create table if not exists student_db.students(
    	id int auto_increment primary key,
        name varchar(100) not null,
        age int not null,
        scores json
        )
        '''
    )
    print('students 테이블 생성완료')
    # 테이블 구조 확인
    cursor.execute('desc students');
    print(f'테이블 구조 : {cursor.fetchall()}')
    conn.commit()
except Exception as e:
    print(e)
finally:
    if conn:              # 생성의 역순으로 닫아야함
        cursor.close()
        conn.close()

print(result)

DB success connection
students 테이블 생성완료
테이블 구조 : [('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(100)', 'NO', '', None, ''), ('age', 'int', 'NO', '', None, ''), ('scores', 'json', 'YES', '', None, '')]
[{'id': 1, 'name': 'Kabul', 'ct': 'AFG', 'dt': 'Kabol', 'pp': 1780000}, {'id': 2, 'name': 'Qandahar', 'ct': 'AFG', 'dt': 'Qandahar', 'pp': 237500}, {'id': 3, 'name': 'Herat', 'ct': 'AFG', 'dt': 'Herat', 'pp': 186800}, {'id': 4, 'name': 'Mazar-e-Sharif', 'ct': 'AFG', 'dt': 'Balkh', 'pp': 127800}, {'id': 5, 'name': 'Amsterdam', 'ct': 'NLD', 'dt': 'Noord-Holland', 'pp': 731200}, {'id': 6, 'name': 'Rotterdam', 'ct': 'NLD', 'dt': 'Zuid-Holland', 'pp': 593321}, {'id': 7, 'name': 'Haag', 'ct': 'NLD', 'dt': 'Zuid-Holland', 'pp': 440900}, {'id': 8, 'name': 'Utrecht', 'ct': 'NLD', 'dt': 'Utrecht', 'pp': 234323}, {'id': 9, 'name': 'Eindhoven', 'ct': 'NLD', 'dt': 'Noord-Brabant', 'pp': 201843}, {'id': 10, 'name': 'Tilburg', 'ct': 'NLD', 'dt': 'Noord-Brabant', 'pp': 193238}]


In [ ]:
# 학생 테이블에서 대해서 CRUD 구현
# 추가기능

# table 생성
args = {
    'host' : 'localhost',
    'user' : 'root',
    'password' : 'root1234',
    'port' : 3306,
    'database' : 'student_db'   # 접속할 때 활성화할 db(스키마)명
}

# DB 접속 함수
# 파라미터 : 접속정보
# 리턴값 : 접속객체

def get_connection(args):
    return connector.connect(**args)   # localhost에 접속 해서 접속 객체를 생성

import json
def add_student(name, age, scores):
    conn = None  # Initialize conn to None
    curson = None
    try:
        conn = connector.connect(**args)  # Unpack args dictionary
        cursor = conn.cursor()
        query = f'insert into student_db.students(name, age, scores) values(%s, %s, %s)'
        cursor.execute(query, (name, age, json.dumps(scores)))
        conn.commit()
        return True
    except Exception as e:
        print(e)
        return True
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
get_connection(args)
add_student('이성계', 25, {'kor':95,'eng':95,'math':100})

True

In [ ]:
# 학생 정보 조회
def get_student(name = None):
    '''
    parameter : name은 기본값이 None , 원하는 이름으로 조회
    return : 성공 = 조회결과, 실패 = False
    '''
    conn = None
    cursor = None
    try:
        conn = get_connection(args)   # 접속
        cursor = conn.cursor()        # 쿼리실행 객체
        query = "select * from students"   # 쿼리 문장
        if name:
            query = f"select * from students where name = '{name}'"
        cursor.execute(query)   # 쿼리 실행
        students = cursor.fetchall()       # 실행 한 쿼리의 리턴값(없으면 None)
        return students
    except Exception as e:
        print(e)
        return True
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
get_student('이성계')

[(1, '이성계', 25, '{"eng": 95, "kor": 95, "math": 100}'),
 (3, '이성계', 25, '{"eng": 95, "kor": 95, "math": 100}')]

In [ ]:
def update_student(id, name=None, age=None, scores=None):
    '''
    update 테이블명 set 컬럼명=변경될값, 컬럼명=변경될 값 where 조건컬럼 =조건값

    parameter : name은 기본값이 None , 원하는 값만 업데이트
    return : 성공 = 조회결과, 실패 = False
    '''
    conn = None
    cursor = None
    updates, params = [],[]
    try:
        conn = get_connection(args)   # 접속
        cursor = conn.cursor()        # 쿼리실행 객체
        if name:
            updates.append("name = %s")
            params.append(name)
        if age:
            updates.append("age = %s")
            params.append(age)
        if scores:
            updates.append('scores = %s')
            params.append(json.dumps(scores))
        merged = ','.join(updates)
        query = "update * from students set"
        query = f'update students set {','.join(updates)} where id ={id}' # 쿼리 문장
        print(query)   # 디버깅용
        print(params)
        cursor.execute(query, params)   # 쿼리 실행
        conn.commit()   # 실제 db에 반영
        return True
    except Exception as e:
        print(e)
        return True
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
update_student(1, name='장보고', age=40, scores={"eng": 70, "kor":90, "math":80 })

update students set name = %s,age = %s,scores = %s where id =1
['장보고', 40, '{"eng": 70, "kor": 90, "math": 80}']


True

In [ ]:
# 삭제
query= 'delete from students where id = %s'
def delete_student(student_id:list):
    conn,cursor = None,None
    try:
        conn = get_connection(args)
        cursor = conn.cursor()
        cursor.execute('delete from students where id = %s',student_id)
        conn.commit()
        print('삭제완료')
    except Exception as e:
        print(f'삭제오류 : {e}')
        return False
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
delete_student([2])

삭제완료


# 정리